In [2]:

# %%
import sys
sys.path.append('../')


from typing import Final

# from helpers.ploting import plotBargraph, plotSubBarGraphs
from helpers.graphs import intersection_of_graphs

# from helpers.operator_actions import __getTrueAndNuisanceSources, __getFinalOperatorAlarmRelationGraph, getTrueAndNuisanceSourceNames
from helpers.alarms import  filterAlarmData, getDFWithCommonSourcesInAllMonths, loadAlarmsData, loadOperatorData
from helpers import alarms
import time
from datetime import datetime, timedelta
from multiprocessing import Pool
import networkx as nx

# from multiprocessing import Pool

class Action2Alarm: # a class which relate operator actions to most suitable corresponding alarms 
    def __init__(self):
        # print("class Created")
        pass

    def __checkAction_OnAlarm(self, action, alarm,time_to_react): # Ahmet Can time_to_react filter # Done
        if action["EventTime"] >= alarm["StartTime"] and action["EventTime"] <= alarm["EndTime"] and timedelta.total_seconds(action["EventTime"]-alarm['StartTime']) <=time_to_react:
            return True
        else:
            return False

    def __addOrUpdateEdge(self,g, e): # e = (op, alarm) # Done
        if g.has_edge(*e) == True:
            g.edges[e]["weight"] += 1
        else:
            g.add_edge(*e, weight=1)
    
    def __updateNode(self, g, node):  # using partial thats why node as last arg
        # if operatorf == True:
        #     node = "Operator->" + node
        # g.nodes[node]["size"] += 0.01
        g.nodes[node]["count"] += 1
        
    def constructSingleAlarmsActionsGraph(self,args): # Done  # case 3
        df_alarms  = args[0] # alarms
        df_actions = args[1] # operator actions
        time_to_react = args[2] # time to react 
        # edge_drop_factor = args[3]  
        g = nx.DiGraph()  # Directed graph
    

        # ---------- Adding Nodes----------------
        alarm_nodes =   [alarm for alarm in df_alarms['SourceName'].unique()]
        action_nodes = ["Operator->"+action for action in df_actions['SourceName'].unique()]     

        _ = [g.add_node(node, size=0, count=0) for node in alarm_nodes+action_nodes]        
        _ = [self.__updateNode(g,node) for node in df_alarms["SourceName"]]
        _ = [self.__updateNode(g,"Operator->"+node) for node in df_actions["SourceName"]]

        # -------- Adding Edges -----------------
        alarms = df_alarms.to_dict(orient="records")
        actions = df_actions.to_dict(orient="records")

        edges = [("Operator->"+action["SourceName"], alarm["SourceName"]) for action in actions for alarm in alarms if self.__checkAction_OnAlarm(action, alarm,time_to_react)]
        
        _ = [self.__addOrUpdateEdge(g,e) for e in edges]

        # if edge_drop_factor > -1:
        #     g = filterEdges(g, edge_drop_factor=edge_drop_factor)

        return g

    def __constructMultipleAlarmsActionsGraphs(self,df_alarms, df_actions,time_to_react,edge_drop_factor):
        # graphs = [self.constructSingleAlarmsActionsGraph((df_alarms[df_alarms["Year-Month"].isin([month])],df_actions[df_actions["Year-Month"].isin([month])],time_to_react,edge_drop_factor)) for month in  df_alarms["Year-Month"].unique()]
        graphs = []

        for month in  df_alarms["Year-Month"].unique():
            inputs = (df_alarms[df_alarms["Year-Month"].isin([month])],df_actions[df_actions["Year-Month"].isin([month])],time_to_react,edge_drop_factor)
            temp_g =  self.constructSingleAlarmsActionsGraph(inputs)
            graphs.append(temp_g)

        return graphs
    
    def getOperatorAlarmRelationGraph(self, df_alarms, df_actions, min_graphs_intersection_filter,time_to_react,edge_drop_factor):
        # print(">> Finding relation between operator action and alarm")
        graphs = self.__constructMultipleAlarmsActionsGraphs(
            df_alarms, df_actions,time_to_react,edge_drop_factor)
            
        assert len(graphs) == len(list(df_alarms["Year-Month"].unique()))
        
        # print(f"\n\n\n\n>> Taking intersection of {len(graphs)} sub-graphs")
        G = intersection_of_graphs(graphs, min_graphs_intersection_filter) # min_graphs_intersection_filter is the gamma filter from the paper 

        return G
    
    # def printGraph(self,g):
    #     print("          ========================== start printing        ================================")
    #     main_g = g
    #     operator_nodes = [action for action in g.nodes if action.find("Operator") != -1]
    #     for op in operator_nodes:
    #         action_count = main_g.nodes[op]['count']
    #         num_neg = len(list(main_g.neighbors(op)))
    #         print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
    #         # s = ""
    #         for n in main_g.neighbors(op):
    #             alarm_count = main_g.nodes[n]['count']
    #             edge_weight = main_g.edges[(op,n)]['weight']
    #             action_alarms_ratio = alarm_count/action_count


    #             print(f"{n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})", end = ", ")
        
    #     print("\n           ========================== end               ================================")


    # def __getTrueAndNuisanceSources(self,df,g):
    #     source_names_requried_action = [
    #         source for source in g.nodes if source.find("Operator") == -1]
    #     df_temp = df[~df["SourceName"].isin(source_names_requried_action)]
    #     sources_not_require_action = df_temp["SourceName"].unique()
    #     return source_names_requried_action, sources_not_require_action
    

    # def getTrueAndNuisanceSourceNames(self, df_alarms,df_operator, num_sub_graphs, min_intersection_f,edge_filter):
        
    #     sources_need_action, sources_dont_need_action = self.__getTrueAndNuisanceSources(df_alarms, g=filterEdges(self.__getFinalOperatorAlarmRelationGraph(df_alarms=df_alarms, df_actions=df_operator,
    #                                                 num_graphs=num_sub_graphs, min_graphs_intersection_filter=min_intersection_f), edge_drop_factor=edge_filter))
    #     return sources_need_action, sources_dont_need_action
    
        
    # def __addOrUpdateEdge(self,g, e):
    #     if g.has_edge(*e) == True:
    #         g.edges[e]["weight"] += 1
    #     else:
    #         g.add_edge(*e, weight=1)


    #     return False



In [3]:

# /home/waris/Github/research/tupras-analysis/data/processed/alarms/final/final-all-months-alarms.csv
"""  Lodading the Data and Preprocessing """

PATH = "/Users/hamzagorgulu/Desktop/thesis/Waris Final/tupras/data/"

alarm_file_path = PATH + "final-all-months-alarms-with-day-filtered.csv"
op_action_file_path = PATH + "final-all-month-actions-filtered.csv"

start = time.time()

df_temp_alarms_main =loadAlarmsData(file_path=alarm_file_path)
df_temp_actions = loadOperatorData(file_path=op_action_file_path)



""" Common Sources in all months. Try it but can be skipped. """
# df_main_alarms = getDFWithCommonSourcesInAllMonths(df_main_alarms)

""" Chaning name 2 alias for alarm data but skipping it """
# source2Alias, alias2source = convertSourceNamesToAlias(df_main_alarms)

print("Total Time to load the data ", time.time()-start)
df_temp_alarms_main
df_temp_alarms_main.info()




Total Time to load the data  43.360414266586304
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12222300 entries, 0 to 12222299
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   Message      object        
 3   Condition    object        
 4   MessageType  object        
 5   StartTime    datetime64[ns]
 6   EndTime      datetime64[ns]
 7   EndMessage   object        
 8   TimeDelta    float64       
 9   Year-Month   object        
 10  Day          int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 1.0+ GB


In [4]:
df_temp_alarms_main.drop_duplicates(keep=False,inplace=True)
df_temp_alarms_main.info()
df_temp_actions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12221979 entries, 0 to 12222299
Data columns (total 11 columns):
 #   Column       Dtype         
---  ------       -----         
 0   MachineName  object        
 1   SourceName   object        
 2   Message      object        
 3   Condition    object        
 4   MessageType  object        
 5   StartTime    datetime64[ns]
 6   EndTime      datetime64[ns]
 7   EndMessage   object        
 8   TimeDelta    float64       
 9   Year-Month   object        
 10  Day          int64         
dtypes: datetime64[ns](2), float64(1), int64(1), object(7)
memory usage: 1.1+ GB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104700 entries, 0 to 104699
Data columns (total 4 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   MachineName  104700 non-null  object        
 1   SourceName   104700 non-null  object        
 2   EventTime    104700 non-null  datetime64[ns]
 3   Year-Month

In [5]:
df_temp_actions.drop_duplicates(keep=False, inplace=True)
df_temp_actions

,MachineName,SourceName,EventTime,Year-Month
0,PHD47B,47FIC017,2020-01-02 14:11:30,"(2020, 1)"
1,PHD47B,47FIC017,2020-01-02 14:46:05,"(2020, 1)"
2,PHD47B,47FIC013,2020-01-02 08:54:53,"(2020, 1)"
3,PHD47B,47FIC021,2020-01-02 08:20:01,"(2020, 1)"
4,PHD47B,47FIC013,2020-01-02 09:09:30,"(2020, 1)"
...,...,...,...,...
104695,PHD47B,48FFX005,2020-02-26 05:46:24,"(2020, 2)"
104696,PHD47B,47TIC927E,2020-02-28 19:09:33,"(2020, 2)"
104697,PHD47B,48TIC034,2020-02-26 00:16:03,"(2020, 2)"
104698,PHD47B,47TIC927E,2020-02-28 19:41:28,"(2020, 2)"


In [6]:
common_df = df_temp_alarms_main[df_temp_alarms_main['SourceName'].isin(df_temp_actions['SourceName'].unique())]
common_df_sources = common_df['SourceName'].unique()
common_df_sources 

array(['48TIC2026', '47FLI-032', '47PDI1746', '47FIC1535', '47PIC1755',
       '47LIC1555', '48XL001-ANN', '47PIC1507', '47LIC1520', '47LIC1506',
       '47FIC1522A', '48AIC001', '47CEMS-SO2N', '48LI013', '47LIC002',
       '47LIC015', '47FI1575', '47PI1710', '47LIC1501', '47LIC1518',
       '48LI011', '48TIC034', '47TI1521', '47FI046', '47LIC1509',
       '47TI1506', '47FIC1551', '47LIC1513', '47FIC1516', '47TI416',
       '47LI024', '47FIC1543', '47TI414G', '48LIC009', '48LI015',
       '48LIC002', '47TI1769', '47FIC1534', '47TI414I', '47LIC1522',
       '47TI1551', '47TIC1535', '47PIC1505', '47FI1509', '47FIC035',
       '47TI1533', '47FIC1520', '47PDIC1759', '47LIC1507', '47LIC1505',
       '47TI1540', '47TI1705', '47FIC1523', '47TIC1551', '47FIC1503',
       '47PIC1560', '47TI414L', '47PIC1712', '47FIC1504', '47TI1722',
       '47TI414P', '47FIC1502', '47FIC012', '47TI1507', '47TI1505',
       '47FIC016', '47FIC038', '47FIC021', '47TI1724', '47FIC020',
       '47FIC1515', '47PI100

In [7]:
df_temp_alarms_main[df_temp_alarms_main['SourceName'].isin(['47PIC2038_Y'])]

,MachineName,SourceName,Message,Condition,MessageType,StartTime,EndTime,EndMessage,TimeDelta,Year-Month,Day


In [8]:
df_temp_actions[df_temp_actions['SourceName'].isin(['47PIC2038_Y'])]

,MachineName,SourceName,EventTime,Year-Month


In [9]:
"""
    alarms -> operator
    47TI1521 -> 47HIC1504, 47HIC1503
    47TI1512C -> (47FIC017, wrong) (47TIC1511, possible correct)
    47LIC1509 -> 47FIC1523, 47LIC1510, 47LIC1509
    47LIC3408 -> 47LIC3404
    47LIC1503 -> 47FIC1507
    47TIC1511 -> 47FIC1502/3/4/5

    47TI2058B Count:79|| Corresponding Actions = 13, => (**47TI2058B, count:114,weight:43, alarm/action: 0.69, weight/count: 0.54)  
    47LIC1505 Count:216|| Corresponding Actions = 3, => (**47LIC1505, count:1357,weight:266, alarm/action: 0.16, weight/count: 1.23)  
    47TI2057A Count:100|| Corresponding Actions = 3, => (**47TI2057A, count:113,weight:45, alarm/action: 0.88, weight/count: 0.45)  
    47FIC3410 Count:288|| Corresponding Actions = 6, => (**47FIC3410, count:528,weight:100, alarm/action: 0.55, weight/count: 0.35)  
    47TI2058A Count:49|| Corresponding Actions = 19, => (**47TI2058A, count:60,weight:30, alarm/action: 0.82, weight/count: 0.61)  
    47FIC020 Count:104|| Corresponding Actions = 17, => (**47FIC020, count:369,weight:162, alarm/action: 0.28, weight/count: 1.56)  
    47PIC027 Count:96|| Corresponding Actions = 1, => (**47PIC027, count:4496,weight:342, alarm/action: 0.02, weight/count: 3.56)  
    47PIC1504 Count:59|| Corresponding Actions = 1, => (**47PIC1504, count:476,weight:180, alarm/action: 0.12, weight/count: 3.05)
    19LIC104 Count:157|| Corresponding Actions = 9, => (**19LIC104, count:308,weight:102, alarm/action: 0.51, weight/count: 0.65)  
    48FFIC019 Count:85|| Corresponding Actions = 1, => (**48FFIC019, count:591,weight:51, alarm/action: 0.14, weight/count: 0.6)
    48LI015 Count:442|| Corresponding Actions = 8, => (**48LI015, count:1457,weight:456, alarm/action: 0.3, weight/count: 1.03)  
    48LI011 Count:759|| Corresponding Actions = 26, => (**48LI011, count:91,weight:56, alarm/action: 8.34, weight/count: 0.07)
    47LIC002 Count:328|| Corresponding Actions = 5, => (**47LIC002, count:565,weight:150, alarm/action: 0.58, weight/count: 0.46)  
    47LIC015 Count:132|| Corresponding Actions = 1, => (**47LIC015, count:191,weight:73, alarm/action: 0.69, weight/count: 0.55)
    47LIC1509 Count:335|| Corresponding Actions = 9, => (47FIC1523, count:781,weight:361, alarm/action: 0.43, weight/count: 1.08) (47LIC1510, count:448,weight:169, alarm/action: 0.75, weight/count: 0.5) (**47LIC1509, count:750,weight:123, alarm/action: 0.45, weight/count: 0.37)
    48TIC034 Count:1971|| Corresponding Actions = 22, => (47TIC937E, count:3867,weight:605, alarm/action: 0.51, weight/count: 0.31) (47TIC927E, count:2940,weight:531, alarm/action: 0.67, weight/count: 0.27) (47PIC3522, count:3165,weight:854, alarm/action: 0.62, weight/count: 0.43) (47TIC007, count:3026,weight:561, alarm/action: 0.65, weight/count: 0.28) (47TIC259, count:2908,weight:511, alarm/action: 0.68, weight/count: 0.26) (47LIC3103, count:2783,weight:970, alarm/action: 0.71, weight/count: 0.49) (48FIC007, count:877,weight:523, alarm/action: 2.25, weight/count: 0.27) (48TIC008, count:1429,weight:510, alarm/action: 1.38, weight/count: 0.26) (**48TIC034, count:1872,weight:1567, alarm/action: 1.05, weight/count: 0.8) (47FIC3102, count:2280,weight:718, alarm/action: 0.86, weight/count: 0.36) 
    47PIC2033A Count:1374|| Corresponding Actions = 48, => (47FIC017, count:2156,weight:387, alarm/action: 0.64, weight/count: 0.28) (47FIC1520, count:2020,weight:432, alarm/action: 0.68, weight/count: 0.31) (47PIC2033B, count:2315,weight:456, alarm/action: 0.59, weight/count: 0.33) (47TIC3520, count:2745,weight:539, alarm/action: 0.5, weight/count: 0.39) (**47PIC2033A, count:2410,weight:466, alarm/action: 0.57, weight/count: 0.34) (47TIC927E, count:2940,weight:585, alarm/action: 0.47, weight/count: 0.43) (47PIC3522, count:3165,weight:640, alarm/action: 0.43, weight/count: 0.47) (47TIC007, count:3026,weight:585, alarm/action: 0.45, weight/count: 0.43) (47TIC259, count:2908,weight:545, alarm/action: 0.47, weight/count: 0.4) (47HIC1502, count:1809,weight:362, alarm/action: 0.76, weight/count: 0.26) (47HIC1501, count:1773,weight:353, alarm/action: 0.77, weight/count: 0.26) (47FIC2002, count:2351,weight:487, alarm/action: 0.58, weight/count: 0.35) (47HIC2008, count:2991,weight:625, alarm/action: 0.46, weight/count: 0.45) (47LIC3103, count:2783,weight:674, alarm/action: 0.49, weight/count: 0.49) (47LIC3404, count:1674,weight:394, alarm/action: 0.82, weight/count: 0.29) (47FIC3102, count:2280,weight:542, alarm/action: 0.6, weight/count: 0.39)  
    47PIC3522 Count:1164|| Corresponding Actions = 35, => (47FIC017, count:2156,weight:307, alarm/action: 0.54, weight/count: 0.26) (47TIC3520, count:2745,weight:656, alarm/action: 0.42, weight/count: 0.56) (**47PIC3522, count:3165,weight:1524, alarm/action: 0.37, weight/count: 1.31) (47LIC3404, count:1674,weight:534, alarm/action: 0.7, weight/count: 0.46) (47FIC3102, count:2280,weight:422, alarm/action: 0.51, weight/count: 0.36)
    47FIC1535 Count:1227|| Corresponding Actions = 34, => (**47FIC1535, count:4930,weight:3601, alarm/action: 0.25, weight/count: 2.93) (47FIC1534, count:1528,weight:959, alarm/action: 0.8, weight/count: 0.78) (47FIC013, count:1786,weight:491, alarm/action: 0.69, weight/count: 0.4) (47FIC1542, count:1838,weight:598, alarm/action: 0.67, weight/count: 0.49) (47TIC937E, count:1674,weight:508, alarm/action: 0.73, weight/count: 0.41) (47TIC274, count:2318,weight:779, alarm/action: 0.53, weight/count: 0.63) (47TIC3520, count:1666,weight:521, alarm/action: 0.74, weight/count: 0.42) (47TIC927E, count:1301,weight:409, alarm/action: 0.94, weight/count: 0.33) (47PIC3522, count:1788,weight:446, alarm/action: 0.69, weight/count: 0.36) (47TIC007, count:1278,weight:334, alarm/action: 0.96, weight/count: 0.27) (47TIC259, count:1306,weight:477, alarm/action: 0.94, weight/count: 0.39) (47FIC1563, count:2227,weight:796, alarm/action: 0.55, weight/count: 0.65) (47HIC2008, count:1521,weight:532, alarm/action: 0.81, weight/count: 0.43) (47HIC2004, count:1048,weight:357, alarm/action: 1.17, weight/count: 0.29) (47PIC1507, count:660,weight:481, alarm/action: 1.86, weight/count: 0.39)
    47TIC3520 Count:2969|| Corresponding Actions = 37, => (47TIC274, count:5269,weight:783, alarm/action: 0.56, weight/count: 0.26) (**47TIC3520, count:2745,weight:1515, alarm/action: 1.08, weight/count: 0.51) (47PIC3522, count:3165,weight:1071, alarm/action: 0.94, weight/count: 0.36) (47LIC3404, count:1674,weight:819, alarm/action: 1.77, weight/count: 0.28) (47PIC027, count:4674,weight:813, alarm/action: 0.64, weight/count: 0.27)
"""  

'\n    alarms -> operator\n    47TI1521 -> 47HIC1504, 47HIC1503\n    47TI1512C -> (47FIC017, wrong) (47TIC1511, possible correct)\n    47LIC1509 -> 47FIC1523, 47LIC1510, 47LIC1509\n    47LIC3408 -> 47LIC3404\n    47LIC1503 -> 47FIC1507\n    47TIC1511 -> 47FIC1502/3/4/5\n\n    47TI2058B Count:79|| Corresponding Actions = 13, => (**47TI2058B, count:114,weight:43, alarm/action: 0.69, weight/count: 0.54)  \n    47LIC1505 Count:216|| Corresponding Actions = 3, => (**47LIC1505, count:1357,weight:266, alarm/action: 0.16, weight/count: 1.23)  \n    47TI2057A Count:100|| Corresponding Actions = 3, => (**47TI2057A, count:113,weight:45, alarm/action: 0.88, weight/count: 0.45)  \n    47FIC3410 Count:288|| Corresponding Actions = 6, => (**47FIC3410, count:528,weight:100, alarm/action: 0.55, weight/count: 0.35)  \n    47TI2058A Count:49|| Corresponding Actions = 19, => (**47TI2058A, count:60,weight:30, alarm/action: 0.82, weight/count: 0.61)  \n    47FIC020 Count:104|| Corresponding Actions = 17, =>

In [10]:
df_temp_alarms = df_temp_alarms_main

# df_temp_alarms = df_temp_alarms[df_temp_alarms["SourceName"].isin(["47TI1512C"])]

# df_temp_alarms = df_temp_alarms[df_temp_alarms["SourceName"].isin(["47TI1521","47TI1512C","47LIC3408","47LIC1509","47LIC1503", '47TIC1511'])]

# df_temp_actions = df_temp_actions[df_temp_actions["SourceName"].isin(["47FIC1502","47FIC1503","47FIC1504","47FIC1505","47FIC1523","47FIC1507"])]

# df_temp_actions = df_temp_actions[df_temp_actions["Year-Month"].isin(df_temp_alarms['Year-Month'].unique())]


#actions "47FIC1502","47FIC1503","47FIC1504","47FIC1505" #,"47FIC1523","47FIC1507"

# "47FIC1502","47FIC1503","47FIC1504","47FIC1505","47FIC1523"

#alarms "47TIC1511","47LIC1503",'47LIC1509'

# df_temp_alarms = df_temp_alarms.sort_values(by='StartTime')
# df_temp_actions =  df_temp_actions.sort_values(by='EventTime')

# df_temp_actions.to_csv("temp_actions.csv", index = False)
# df_temp_alarms.to_csv("temp_alarms.csv", index = False)



In [11]:
print("Alarms >",df_temp_alarms["SourceName"].value_counts())

print("Actions >", df_temp_actions["SourceName"].value_counts())

Alarms > 48TIC2026          7162316
47TI931A           3285328
47FLI-032           294110
48PAL007-ANN        250472
47TI003A            124399
                    ...   
47HCD-080HH-ANN          1
47HSD-030HH-ANN          1
47HCD-061HH-ANN          1
47HCD-063HH-ANN          1
47FI033C                 1
Name: SourceName, Length: 1090, dtype: int64
Actions > 47TIC274           5269
47FIC1535          4985
47PIC027           4674
47FIC013           4069
47FIC1563          3903
                   ... 
47APC_RCAS_SHDF       1
47FIC1503_DSET        1
47FIC1504_DSET        1
47FIC1505_DSET        1
47APC_F21_LC          1
Name: SourceName, Length: 456, dtype: int64


In [41]:
# temp_obj = Action2Alarm()
# test_g =  temp_obj.constructSingleAlarmsActionsGraph((df_temp_alarms,df_temp_actions,0.5*60*60))

# main_g = test_g
# # %%
# operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]

# dict_to_print = {}
# for op in operator_nodes:
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     s = ""
#     for n in main_g.neighbors(op):
#         alarm_count = main_g.nodes[n]['count']
#         edge_weight = main_g.edges[(op,n)]['weight']
#         action_alarms_ratio = alarm_count/action_count

#         s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), "
#         dict_to_print[op] = s    


# for op in dict_to_print.keys():
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg}, => {dict_to_print[op]} " )

# # temp_obj.printGraph(main_g)

In [42]:

# temp_obj = Action2Alarm()
# # test_g =  temp_obj.constructSingleAlarmsActionsGraph((df_temp_alarms,df_temp_actions,2.5*60*60))
# test_g =  temp_obj.getOperatorAlarmRelationGraph(df_alarms=df_temp_alarms, df_actions=df_temp_actions, min_graphs_intersection_filter=4, time_to_react=8*60*60)

# main_g = test_g
# operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]

# dict_to_print = {}
# for op in operator_nodes:
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     # print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
#     s = ""
#     for n in main_g.neighbors(op):
#         alarm_count = main_g.nodes[n]['count']
#         edge_weight = main_g.edges[(op,n)]['weight']
#         action_alarms_ratio = alarm_count/action_count

#         s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), "
#         dict_to_print[op] = s    

# for op in dict_to_print.keys():
#     action_count = main_g.nodes[op]['count']
#     num_neg = len(list(main_g.neighbors(op)))
#     print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg}, => {dict_to_print[op]} " )



In [12]:
def filterEdges(g, edge_drop_factor):
    # g = nx.DiGraph(g)
    remove_edges = []
    for source, target, weight in g.edges.data("weight"):
        alarm_count = g.nodes[target]["count"]
        action_count = g.nodes[source]["count"]

        alarms2action_ratio = alarm_count/action_count
        weight_alarm_ratio =  weight/alarm_count

        # if edge_drop_factor * weight < g.nodes[target]["count"]:
        if  alarms2action_ratio <  0.20:
            remove_edges.append((source, target))
        elif alarms2action_ratio <0.10 and weight_alarm_ratio < 0.25:
            remove_edges.append((source, target))

    g.remove_edges_from(remove_edges)
    g.remove_nodes_from(list(nx.isolates(g)))
    return g

def foo1(args):

        df_alarms = args[0]
        df_actions =args[1]
        min_intersect = args[2]
        time_to_react = args[3]
        edge_drop_factor = args[4]
        actionAndAlarms = []

        action2alarm = Action2Alarm()

        # print(f'Months in Alarms = {df_alarms["Year-Month"].unique()} \n Months in Actions = {df_actions["Year-Month"].unique()}')

        main_g = action2alarm.getOperatorAlarmRelationGraph(df_alarms=df_alarms, df_actions=df_actions, min_graphs_intersection_filter=min_intersect, time_to_react=time_to_react,edge_drop_factor=edge_drop_factor)

            # %%
        # operator_nodes = [action for action in main_g.nodes if action.find("Operator") != -1]
        alarm_nodes = [alarm for alarm in main_g.nodes if alarm.find("Operator") == -1]

        # print(f">> # of Edges Main Graph {main_g.number_of_edges()}")
        # print(f">> Total number of operator Tags in the graph={len(operator_nodes)}")
        # print(f">> Total number of Alarm Tags in the graph = {len(main_g.nodes)- len(operator_nodes)}")

        dict_to_print = {}
        for node in alarm_nodes:
            alarm_count = main_g.nodes[node]['count']
            num_neg = len(list(main_g.predecessors(node)))
            # print(f"{op} Count:{action_count}|| Correspodning Alarms = {num_neg} ", end = " " )
            s = ""
            for op_node in main_g.predecessors(node):
                # print(op_node)
                act_count = main_g.nodes[op_node]['count']
                edge_weight = main_g.edges[(op_node,node)]['weight']
                action_alarms_ratio = round(alarm_count/act_count,2)
                weight_alarm_ratio =  round(edge_weight/alarm_count,2)

                # s = s+ f" {n}(count:{alarm_count})(weight:{edge_weight}), (alarm/action: {action_alarms_ratio}), (alarm/weight: {action_weight_ratio}) "
                # dict_to_print[op] = s 
                op_node = op_node.replace("Operator->","")
                if op_node == node:
                    s = s+ f"(**{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s 
                elif action_alarms_ratio > 0.4 and weight_alarm_ratio>0.25:
                    s = s+ f"(2-{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s
                elif action_alarms_ratio>0.3 and weight_alarm_ratio > 0.8: # these ratios are not finalized and final ratios will depend on the Ahmet Can feedback
                    s = s+ f"(3-{op_node}, count:{act_count},weight:{edge_weight}, alarm/action: {action_alarms_ratio}, weight/alarm: {weight_alarm_ratio}) "
                    dict_to_print[node] = s  
                # elif action_alarms_ratio >0.5 and action_alarms_ratio <=1:
                #     print(f"{n}(count:{alarm_count})(weight:{edge_weight}), (ratio: {action_alarms_ratio})", end = ", ")


        for alarm_node in dict_to_print.keys():
            alarm_count = main_g.nodes[alarm_node]['count']
            num_neg = len(list(main_g.predecessors(alarm_node)))

            s = f"{alarm_node} Count:{alarm_count}|| Corresponding Actions = {num_neg}, => {dict_to_print[alarm_node]} " 
            print(s)
            actionAndAlarms.append(s)
        
        return actionAndAlarms

In [13]:
print(f"Total Alarms: {df_temp_alarms.shape}, Total Operator Action: {df_temp_actions.shape}, Total Sources: {len(df_temp_alarms['SourceName'].unique())} ")

# %%

""" 
    Filter the Alarm Data
    1. Ignore the communication Alarms
    2. Ignore the momentary alarms => 20 seconds
    3. Remove Staling Alarms => 12 hours    
    4. Remove sources which are triggered less 20 in whole dataset
    5. Include all the months
    6. DO SKIP ANY SOURCENAME IF IGNORING COMMUNICATION ALARMS
"""
ignore_comm_alarms :Final = True
momentary_alarms_f:Final = 2*60  # seconds
staling_alarm_f:Final = (60*60) * 4 # hours
min_alarms_per_source_f:Final = 40 # any source which is not triggered atleast 20 times in whole dataset will be removed
months_f:Final = df_temp_alarms["Year-Month"].unique()
print(f">> Unique Year-Month {months_f}")
snames_f:Final = [] # ONLY USE IF NOT IGNORING COMM ALRMS


df_alarms_new = filterAlarmData(df_temp_alarms, months=months_f, sources_filter=snames_f,
                                     monmentarly_filter=momentary_alarms_f, staling_filter=staling_alarm_f, ingore_communication_alarms=ignore_comm_alarms, min_alarms_per_source=min_alarms_per_source_f)

df_actions_new = df_temp_actions[df_temp_actions["Year-Month"].isin(months_f)]

sname2months = {}
for sname in df_alarms_new["SourceName"].unique():
    sname2months[sname] = df_alarms_new[df_alarms_new["SourceName"].isin([sname])]["Year-Month"].unique()



monthCount2Snames = {} 

for key in range(len(list(df_alarms_new["Year-Month"].unique()))):
    monthCount2Snames[key+1] = []

for sname in sname2months.keys():
    monthCount2Snames[len(sname2months[sname])].append(sname)

print(f">> Month Count 2 alarms: {monthCount2Snames}" )

""" 
    For Opertartor Graphs
    1. Construct sub-graphs equal to number of months
    2. Min intersections is sub-graphs/2 + 1
    3. Remove edges which are not significantaly contributing to nodes count
        It will vary in op actions and in grouping. In op actions
        it will be higher but in groupign it will be lower.
"""
# num_sub_graphs:Final = len(months_f)
# min_intersection_f:Final = 2
# edge_filter:Final = 1 # BE CAREFUL OVER HERE

print("Alarms >",df_alarms_new["SourceName"].value_counts())

print("Actions >", df_actions_new["SourceName"].value_counts())

actionAndAlarms =[]
# action2alarm = Action2Alarm()
for sname in  df_alarms_new['SourceName'].unique():
    mc = len(sname2months[sname])
    intersect_threshold = mc-1 # gamma
    if  mc <=5:
        continue
    # print(f">> Source Name {sname}, Alarm Appear in {mc} months, Intersection Threshold ={intersect_threshold}" )
    df1 = df_alarms_new[df_alarms_new['SourceName'].isin([sname])]
    df2 = df_actions_new[df_actions_new['Year-Month'].isin(list(df1['Year-Month'].unique()))] 

    actionAndAlarms += foo1((df1,df2,intersect_threshold,30*60*60, -1))
    # endAlarmTime should be around 1 hour


# for aa in actionAndAlarms:
#     print(f"++{aa}")

# foo1((df_temp_alarms,df_temp_actions,6,4*60*60))

Total Alarms: (12221979, 11), Total Operator Action: (102895, 4), Total Sources: 1090 
>> Unique Year-Month ['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 5)' '(2019, 6)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
>>Preprocessing... 
   Months to include=['(2020, 3)' '(2019, 7)' '(2019, 8)' '(2019, 3)' '(2019, 10)' '(2019, 11)'
 '(2019, 9)' '(2020, 2)' '(2019, 5)' '(2019, 6)' '(2019, 12)' '(2019, 4)'
 '(2020, 1)']
  Ignore Sources=[]
  Ingnore Momentarlily Alarms Filter=120seconds 
   Ignoreing Staling Alarms Filter=4.0 hours, 
 Ignore Communication Alarms = True 
 Remove sources whose count is less than 40
>> Month Count 2 alarms: {1: ['47LI900B', '47LI900ABC'], 2: ['47FIC1543', '47LI850C'], 3: ['47TAH808-ANN', '47PI010', '47FIC011', '47LI900C', '47FIC017'], 4: ['47FLI-032', '47FSL014-ANN', '47FLI-041', '47FI1505D', '48XL011-ANN', '47PIC1712', '47PI1734', '47LI606A', '47PI1519', '47FI1567'], 5: ['47USI802', '48UA014-ANN', '47FI1

In [45]:
"""
    47TI1512C -> 47FIC017, 47FIC013, 47TIC937E, 47TIC274, 47TIC927E, 47TIC007, 47TIC259, 47FIC1563

"""



"""
    alarms -> operator
    47TI1521 -> 47HIC1504, 47HIC1503
    47TI1512C -> (47FIC017, wrong) (47TIC1511, possible correct)
    47LIC1509 -> 47FIC1523, 47LIC1510, 47LIC1509
    47LIC3408 -> 47LIC3404
    47LIC1503 -> 47FIC1507
    47TIC1511 -> 47FIC1502/3/4/5
    
    

47TI1521 Count:397|| Correspodning Actions = 16, => (47HIC1504, count:910,weight:321, alarm/action: 0.44, weight/count: 0.81) (47HIC1503, count:900,weight:331, alarm/action: 0.44, weight/count: 0.83) correct 
47TI1512C Count:1091|| Correspodning Actions = 8, => (47FIC017, count:2156,weight:56, alarm/action: 0.51, weight/count: 0.05)  wrong one, possibly correct: 47TIC1511
47LIC1509 Count:335|| Correspodning Actions = 9, => (47FIC1523, count:781,weight:361, alarm/action: 0.43, weight/count: 1.08) (47LIC1510, count:448,weight:169, alarm/action: 0.75, weight/count: 0.5) (47LIC1509, count:750,weight:123, alarm/action: 0.45, weight/count: 0.37) almost correct
47LIC3408 Count:880|| Correspodning Actions = 3, => (47LIC3404, count:1674,weight:846, alarm/action: 0.53, weight/count: 0.96) correct

"""  

'\n    alarms -> operator\n    47TI1521 -> 47HIC1504, 47HIC1503\n    47TI1512C -> (47FIC017, wrong) (47TIC1511, possible correct)\n    47LIC1509 -> 47FIC1523, 47LIC1510, 47LIC1509\n    47LIC3408 -> 47LIC3404\n    47LIC1503 -> 47FIC1507\n    47TIC1511 -> 47FIC1502/3/4/5\n    \n    \n\n47TI1521 Count:397|| Correspodning Actions = 16, => (47HIC1504, count:910,weight:321, alarm/action: 0.44, weight/count: 0.81) (47HIC1503, count:900,weight:331, alarm/action: 0.44, weight/count: 0.83) correct \n47TI1512C Count:1091|| Correspodning Actions = 8, => (47FIC017, count:2156,weight:56, alarm/action: 0.51, weight/count: 0.05)  wrong one, possibly correct: 47TIC1511\n47LIC1509 Count:335|| Correspodning Actions = 9, => (47FIC1523, count:781,weight:361, alarm/action: 0.43, weight/count: 1.08) (47LIC1510, count:448,weight:169, alarm/action: 0.75, weight/count: 0.5) (47LIC1509, count:750,weight:123, alarm/action: 0.45, weight/count: 0.37) almost correct\n47LIC3408 Count:880|| Correspodning Actions = 3, 

In [14]:
# for aa in actionAndAlarms:
#     print(aa)
with open("output.txt",'w') as f:
    for aa in actionAndAlarms:
        f.write(aa)
        f.write("\n")
    # f.writelines(actionAndAlarms)

In [37]:
from sys import prefix


actionAndAlarms[0].split(" ")[0]  # alarm
actionAndAlarms[20].split(" ")[7].replace("(","").replace(",","") 
prefixes = ["(**", "(2-", "(3-"]

alarm_action_dict = {}
for i in actionAndAlarms:
    i = i.split(" ")
    alarm_action_dict[i[0]] = [x.replace("(","").replace(",","") for x in i if x.startswith(tuple(prefixes))]
print(alarm_action_dict)

{'47TI003A': ['2-47FIC1563'], '47FLI-039': ['2-47FIC013', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', '2-47TIC007', '2-47TIC259', '2-47FIC1563', '2-47PIC027'], '47TI003B': ['2-47TIC274', '2-47FIC1563'], '47FIC1535': ['**47FIC1535', '2-47FIC1534', '2-47FIC013', '2-47FIC1542', '2-47TIC937E', '2-47TIC274', '2-47TIC927E', '2-47TIC007', '2-47TIC259', '2-47FIC1563', '2-47PIC1507'], '48XL001-ANN': ['2-47TIC274'], '47FLI-042': ['2-47FIC1520', '2-47TIC927E', '2-47TIC007', '2-47TIC415', '2-47TIC259', '2-47HIC1502', '2-47HIC1501'], '47FLI-037': ['2-47HIC1502'], '48PAL013-ANN': ['2-48TIC034'], '47XL016-ANN': ['2-47FIC1534', '2-47FIC017', '2-47FIC1520', '3-47FIC013', '2-47FIC035', '2-47FIC036', '2-47FIC1542', '3-47TIC937E', '2-47TIC927E', '2-47TIC007', '2-47TIC415', '2-47TIC259', '2-47HIC1502', '2-47LIC1505', '3-47FIC1563', '2-47HIC1501', '2-47PIC028', '2-47PIC017', '2-48TIC008', '2-48TIC034', '2-48LI015'], '47XL015-ANN': ['2-47FIC1534', '2-47FIC017', '2-47FIC1520', '3-47FIC013', '2-47FIC1542', '3-

In [38]:
# import dict to excel
import pandas as pd
df = pd.DataFrame.from_dict(alarm_action_dict, orient='index')
df.to_excel("output.xlsx")